In [1]:
import os
import re
import io
import sys
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

from dotenv import load_dotenv, find_dotenv
from pathlib import Path

In [2]:
# FUNÇÕES DE PREPARO

def coerce_numeric(s):
    return pd.to_numeric(s, errors="coerce")

def extrair_fase(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if "alfa" in valor:
        return 0
    m = re.search(r"fase\s*(\d+)", valor)
    if m:
        return int(m.group(1))
    return np.nan

def padronizar_genero(df):
    df = df.copy()
    if "genero" in df.columns:
        df["genero"] = df["genero"].astype(str).str.strip().str.lower()
        map_genero = {
            "menino": "masculino",
            "masculino": "masculino",
            "menina": "feminino",
            "feminino": "feminino"
        }
        df["genero"] = df["genero"].map(map_genero)
    return df

def padronizar_idade(df):
    df = df.copy()
    if "idade" not in df.columns:
        return df

    s = df["idade"]
    dt = pd.to_datetime(s, errors="coerce")

    idade_from_date = np.where(
        dt.notna() & (dt.dt.year == 1900) & (dt.dt.month == 1),
        dt.dt.day,
        np.nan
    )

    idade_num = pd.to_numeric(s, errors="coerce")
    idade_final = pd.Series(idade_num, index=df.index)

    mask = idade_final.isna() & ~pd.isna(idade_from_date)
    idade_final.loc[mask] = idade_from_date[mask]

    # faixa plausível
    idade_final = idade_final.where(idade_final.between(6, 30))
    df["idade"] = idade_final.round()
    return df

def tratar_inde_2024(df):
    df = df.copy()
    if "inde_2024" in df.columns:
        tmp = df["inde_2024"].astype(str).str.strip().str.upper()
        tmp = tmp.replace("INCLUIR", np.nan)
        df["inde_2024"] = coerce_numeric(tmp)
    return df

def preparar_base(df, modo_treino: bool):
    """
    modo_treino=True:
      - cria target a partir de 'ian'
      - remove vazamento (ian/defasagem)
    modo_treino=False:
      - não cria target
      - remove vazamento se existir
    """
    df = df.copy()

    df = padronizar_genero(df)
    df = padronizar_idade(df)
    df = tratar_inde_2024(df)

    if "fase_ideal" in df.columns:
        df["fase_ideal"] = df["fase_ideal"].apply(extrair_fase)

    # Target (somente treino)
    if modo_treino:
        if "ian" not in df.columns:
            raise ValueError("modo_treino=True exige coluna 'ian' para criar a target.")
        df["risco_defasagem_atual"] = (pd.to_numeric(df["ian"], errors="coerce") <= 5).astype(int)

    # Remover vazamento sempre que existir
    df = df.drop(columns=[c for c in ["ian", "defasagem"] if c in df.columns], errors="ignore")

    # Remoções iguais ao treino (mantém seu padrão)
    colunas_para_remover = ["ra",
                            "nome",
                            "data_nasc",
                            "escola",
                            "avaliador_1",
                            "avaliador_2",
                            "avaliador_3",
                            "avaliador_4",
                            "avaliador_5",
                            "avaliador_6",
                            "rec_av1",
                            "rec_av2",
                            "rec_av3",
                            "rec_av4",
                            "rec_av5",
                            "rec_av6",
                            "rec_psicologia",
                            "indicado",
                            "atingiu_pv",
                            "destaque_ieg",
                            "destaque_ida",
                            "destaque_ivp",
                            "pedra_2020",
                            "pedra_2021",
                            "pedra_2022",
                            "pedra_2023",
                            "pedra_2024",
                            "fase",
                            "turma",
                            "instituicao_ensino",
                            "ativo_inativo",
                            "cg",
                            "cf",
                            "ct",
                            "inde_2024"
    ]
    df = df.drop(columns=[c for c in colunas_para_remover if c in df.columns], errors="ignore")

    # Features extras
    cols_acad = [c for c in ["mat","por","ing"] if c in df.columns]
    if len(cols_acad) >= 2:
        df["media_academica"] = df[cols_acad].mean(axis=1)

    cols_comp = [c for c in ["iaa","ieg","ips","ipp"] if c in df.columns]
    if len(cols_comp) >= 2:
        df["media_comportamental"] = df[cols_comp].mean(axis=1)

    if ("inde_2022" in df.columns) and ("inde_2023" in df.columns):
        df["delta_inde"] = df["inde_2023"] - df["inde_2022"]

    return df

def make_preprocess(X_train: pd.DataFrame) -> ColumnTransformer:
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_train.columns if c not in num_cols]

    numeric_pipe = Pipeline(steps=[
        ("imputer", KNNImputer(n_neighbors=7, weights="distance")),
        ("scaler", StandardScaler())
    ])

    categorical_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", categorical_pipe, cat_cols),
        ],
        remainder="drop"
    )
    return preprocess

In [3]:
# 2) TREINO COMPLETO + TESTE + THRESHOLD + SALVAR ARQUIVOS
# ============================================================

def treinar_e_salvar(excel_path: str,
                     out_model_path: str = "modelo_passos_magicos.pkl",
                     out_cfg_path: str = "config_passos_magicos.pkl",
                     seed: int = 42):

    base = pd.read_excel(excel_path)
    print("Shape original:", base.shape)

    base2 = preparar_base(base, modo_treino=True)
    print("Shape após preparo:", base2.shape)
    print("Target balance:\n", base2["risco_defasagem_atual"].value_counts())

    if "ano_pede" not in base2.columns:
        raise ValueError("Coluna 'ano_pede' não encontrada após preparo. Precisa dela para split temporal.")

    # 1. Split temporal usando o 'ano_pede'
    train_df = base2[base2["ano_pede"] < 2024].copy()
    test_df  = base2[base2["ano_pede"] == 2024].copy()

    # 2. Separação da Features (X) e Target (y)
    # Remover 'risco_defasagem_atual' e 'ano_pede' para que o algoritmo não aprenda o viés temporal
    colunas_para_ignorar = ["risco_defasagem_atual", "ano_pede"]

    X_train = train_df.drop(columns=colunas_para_ignorar)
    y_train = train_df["risco_defasagem_atual"].astype(int)

    X_test = test_df.drop(columns=colunas_para_ignorar)
    y_test = test_df["risco_defasagem_atual"].astype(int)

    print("Treino:", X_train.shape, "Teste:", X_test.shape)

    preprocess = make_preprocess(X_train)

    models = {
        "LogReg": LogisticRegression(max_iter=2000, class_weight="balanced"),
        "RandomForest": RandomForestClassifier(
            n_estimators=400, random_state=seed,
            class_weight="balanced_subsample"
        ),
        "MLP": MLPClassifier(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            max_iter=500,
            random_state=seed
        ),
    }

    if HAS_XGB:
        models["XGBoost"] = XGBClassifier(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=seed,
            eval_metric="logloss"
        )
    else:
        print("XGBoost não disponível. Seguindo sem XGBoost.")

    # CV no treino
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    scoring = {"acc": "accuracy", "roc_auc": "roc_auc", "pr_auc": "average_precision"}

    print("\n================= CV (TREINO) =================")
    cv_results = []
    pipes = {}

    for name, clf in models.items():
        pipe = Pipeline(steps=[("prep", preprocess), ("model", clf)])
        pipes[name] = pipe

        scores = cross_validate(
            pipe, X_train, y_train,
            cv=cv, scoring=scoring,
            n_jobs=-1, return_train_score=False
        )

        row = {
            "model": name,
            "acc_mean": scores["test_acc"].mean(),
            "acc_std": scores["test_acc"].std(),
            "roc_auc_mean": scores["test_roc_auc"].mean(),
            "roc_auc_std": scores["test_roc_auc"].std(),
            "pr_auc_mean": scores["test_pr_auc"].mean(),
            "pr_auc_std": scores["test_pr_auc"].std(),
        }
        cv_results.append(row)

        print(f"\n{name}")
        print(f"  ACC     : {row['acc_mean']:.4f} ± {row['acc_std']:.4f}")
        print(f"  ROC AUC : {row['roc_auc_mean']:.4f} ± {row['roc_auc_std']:.4f}")
        print(f"  PR AUC  : {row['pr_auc_mean']:.4f} ± {row['pr_auc_std']:.4f}")

    cv_df = pd.DataFrame(cv_results).sort_values("roc_auc_mean", ascending=False)
    print("\nResumo CV:")
    print(cv_df)

    # Teste final 2024
    print("\n================= TESTE FINAL (2024) =================")
    test_rows = []

    def avaliar(nome, pipe):
        pipe.fit(X_train, y_train)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        y_pred = pipe.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba)
        pr  = average_precision_score(y_test, y_proba)

        print(f"\n>>> {nome}")
        print(f"ACC     : {acc:.4f}")
        print(f"ROC AUC : {roc:.4f}")
        print(f"PR AUC  : {pr:.4f}")
        print("\nMatriz de confusão:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification report:")
        print(classification_report(y_test, y_pred, digits=4))

        return {"model": nome, "acc": acc, "roc_auc": roc, "pr_auc": pr}

    for name, pipe in pipes.items():
        test_rows.append(avaliar(name, pipe))

    test_df_res = pd.DataFrame(test_rows).sort_values("roc_auc", ascending=False)
    print("\nResumo TESTE (ordenado por ROC AUC):")
    print(test_df_res)

    best_name = test_df_res.iloc[0]["model"]
    best_pipe = pipes[best_name]
    print("\nMelhor modelo no TESTE:", best_name)

    # Ajuste threshold no MELHOR modelo do teste
    best_pipe.fit(X_train, y_train)
    y_proba_best = best_pipe.predict_proba(X_test)[:, 1]

    thresholds = np.linspace(0.10, 0.90, 17)
    thr_rows = []
    for t in thresholds:
        y_pred_t = (y_proba_best >= t).astype(int)
        report = classification_report(y_test, y_pred_t, output_dict=True, zero_division=0)
        acc = accuracy_score(y_test, y_pred_t)
        prec1 = report["1"]["precision"]
        rec1 = report["1"]["recall"]
        f11 = report["1"]["f1-score"]
        thr_rows.append([t, acc, prec1, rec1, f11])

    thr_df = pd.DataFrame(thr_rows, columns=["threshold","accuracy","precision_risco","recall_risco","f1_risco"])
    print("\nTabela de thresholds (classe RISCO=1):")
    print(thr_df)

    best_thr_row = thr_df.sort_values("f1_risco", ascending=False).iloc[0]
    best_threshold = float(best_thr_row["threshold"])

    print("\nMelhor threshold (por F1 da classe 1):", best_threshold)
    print("Linha escolhida:\n", best_thr_row)

    # Salvar arquivos
    joblib.dump(best_pipe, out_model_path)
    joblib.dump({"threshold": best_threshold, "best_model": best_name}, out_cfg_path)

    print("\nArquivos salvos:")
    print(f"- (pipeline completo)")
    print(f"- (threshold + nome do modelo)")

    return best_name, best_threshold, cv_df, test_df_res, thr_df

In [4]:
# Localiza e carrega as variáveis do arquivo .env automaticamente
load_dotenv(find_dotenv())

# Puxa o caminho da variável de ambiente
data_path = Path(os.getenv("DATA_PATH"))
models_dir = Path(os.getenv("MODELS"))

MODO = "treino"  # "treino" ou "app"
EXCEL_PATH = data_path 
MODEL_PATH = models_dir / "modelo_passos_magicos.pkl"
CFG_PATH = models_dir / "config_passos_magicos.pkl"

if MODO == "treino":
    treinar_e_salvar(EXCEL_PATH, MODEL_PATH, CFG_PATH)

elif MODO == "app":
    # Streamlit NÃO é ideal no Colab, mas em ambiente local funciona:
    # streamlit run seu_arquivo.py
    print("Modo app: rode em ambiente local com Streamlit.")
    print("Exemplo: streamlit run projeto_completo.py")
else:
    raise ValueError("MODO inválido. Use 'treino' ou 'app'.")

Shape original: (3030, 54)
Shape após preparo: (3030, 21)
Target balance:
 risco_defasagem_atual
1    1687
0    1343
Name: count, dtype: int64
Treino: (1874, 19) Teste: (1156, 19)

================= CV (TREINO) =================

LogReg
  ACC     : 0.8949 ± 0.0191
  ROC AUC : 0.9308 ± 0.0216
  PR AUC  : 0.9138 ± 0.0352

RandomForest
  ACC     : 0.9034 ± 0.0086
  ROC AUC : 0.9657 ± 0.0031
  PR AUC  : 0.9752 ± 0.0044

MLP
  ACC     : 0.9589 ± 0.0032
  ROC AUC : 0.9887 ± 0.0013
  PR AUC  : 0.9910 ± 0.0029

XGBoost
  ACC     : 0.9397 ± 0.0071
  ROC AUC : 0.9820 ± 0.0013
  PR AUC  : 0.9877 ± 0.0005

Resumo CV:
          model  acc_mean   acc_std  roc_auc_mean  roc_auc_std  pr_auc_mean  \
2           MLP  0.958911  0.003212      0.988700     0.001299     0.990981   
3       XGBoost  0.939701  0.007081      0.981961     0.001263     0.987693   
1  RandomForest  0.903410  0.008568      0.965735     0.003056     0.975218   
0        LogReg  0.894863  0.019145      0.930814     0.021596     0.91

In [ ]:
pipe = joblib.load(models_dir / "modelo_passos_magicos.pkl")
cfg = joblib.load(models_dir / "config_passos_magicos.pkl")

threshold = cfg['threshold']
print('Threshold: ', threshold)

base = pd.read_excel(data_path)

base_prep = preparar_base(base, modo_treino=False)

sample = base_prep.sample(10, random_state=42)

proba = pipe.predict_proba(sample)[:, 1]
pred = (proba >= threshold).astype(int)

sample['proba'] = proba
sample['pred'] = pred

sample

Threshold:  0.45000000000000007


,ano_pede,idade,genero,ano_ingresso,inde_2022,inde_2023,n_av,iaa,ieg,ips,...,mat,por,ing,ipv,fase_ideal,media_academica,media_comportamental,delta_inde,proba,pred
1207,2023,12.0,masculino,2023,NaN,5.973450,2.0,10.000,9.300000,2.520,...,4.5,3.3,NaN,6.500000,3,3.900000,6.783125,NaN,0.9475,1
256,2024,12.0,feminino,2023,NaN,6.478200,2.0,10.002,5.389610,5.635,...,0.0,4.0,NaN,4.250000,3,2.000000,7.053528,NaN,0.8600,1
2356,2022,12.0,feminino,2019,7.928000,NaN,4.0,8.800,8.900000,7.500,...,5.3,7.2,4.8,7.917000,3,5.766667,8.400000,NaN,0.0775,0
175,2024,9.0,masculino,2024,NaN,NaN,2.0,9.502,10.000000,7.510,...,10.0,6.5,NaN,7.500000,1,8.250000,8.628000,NaN,0.9075,1
211,2024,10.0,masculino,2023,NaN,6.861700,3.0,8.502,9.328063,4.380,...,6.0,8.5,NaN,5.783333,1,7.250000,6.880641,NaN,0.6900,1
411,2024,11.0,feminino,2022,7.038889,5.823133,3.0,9.002,7.595238,4.385,...,3.0,3.0,NaN,5.996667,2,3.000000,6.651810,-1.215756,0.7725,1
52,2024,10.0,feminino,2023,NaN,7.079700,3.0,9.002,9.473684,7.510,...,7.0,7.5,NaN,6.943333,2,7.250000,8.527671,NaN,0.7425,1
266,2024,10.0,masculino,2022,7.736100,7.680200,3.0,10.002,8.484848,6.260,...,10.0,7.0,NaN,8.056667,1,8.500000,8.217962,-0.055900,0.6875,1
479,2024,12.0,feminino,2024,NaN,NaN,4.0,8.502,8.782609,7.510,...,4.0,5.5,NaN,6.827500,3,4.750000,7.917402,NaN,0.3175,0
1292,2023,8.0,feminino,2023,NaN,8.017200,2.0,9.500,8.100000,7.520,...,9.1,8.6,NaN,6.500000,0,8.850000,7.842500,NaN,0.0150,0


In [6]:
list(pipe.named_steps.keys())

['prep', 'model']

In [7]:
model = pipe.named_steps['model']

if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
elif hasattr(model, 'coef_'):
    importances = np.abs(model.coef_[0])
else:
    importances = None
    print("Este modelo não suporta extração de importância de variáveis.")

if importances is not None:
    features = pipe.named_steps['prep'].get_feature_names_out()
    imp = pd.DataFrame({
        'feature': features,
        'importance': importances
    }).sort_values('importance', ascending=False)

    print(imp.head(15))

                      feature  importance
0                  num__idade    0.126111
2              num__inde_2022    0.117895
14            num__fase_ideal    0.117297
4                   num__n_av    0.076756
3              num__inde_2023    0.076359
16  num__media_comportamental    0.048708
17            num__delta_inde    0.044154
15       num__media_academica    0.044071
8                    num__ipp    0.043022
13                   num__ipv    0.039830
10                   num__mat    0.039046
12                   num__ing    0.038428
9                    num__ida    0.037869
11                   num__por    0.034722
6                    num__ieg    0.030362


In [ ]:
def gerar_planilha_validacao():
    
    print("Iniciando a geração da planilha de validação...")
    
    # Busca inteligente do arquivo .env
    load_dotenv(find_dotenv())
    
    # Resgata os caminhos configurados
    data_path = os.getenv("DATA_PATH")
    models_dir = os.getenv("MODELS")
    processed_dir = os.getenv("PROCESSED")
    
    if not all([data_path, models_dir, processed_dir]):
        print("Erro: Verifique se as variáveis DATA_PATH, MODELS e PROCESSED estão corretas no arquivo .env.")
        return
    
    # 1. Carregar modelo e configurações
    caminho_modelo = os.path.join(models_dir, "modelo_passos_magicos.pkl")
    caminho_config = os.path.join(models_dir, "config_passos_magicos.pkl")
    
    if not os.path.exists(caminho_modelo) or not os.path.exists(caminho_config):
        print(f"Erro: Arquivos .pkl não encontrados em {models_dir}.")
        return
        
    modelo = joblib.load(caminho_modelo)
    config = joblib.load(caminho_config)
    limiar = config['threshold']
    
    # 2. Carregar a base de dados original
    if not os.path.exists(data_path):
        print(f"Erro: Base de dados não encontrada em {data_path}")
        return
        
    df_raw = pd.read_excel(data_path)
    
    # =========================================================================
    # NOVO: SELEÇÃO ALEATÓRIA
    # Pega exatamente 5 registros (linhas) aleatórios da base original
    # =========================================================================
    df_raw = df_raw.sample(n=10)
    
    # 3. Preparar a base para passar pelo modelo
    df_processado = preparar_base(df_raw, modo_treino=False)
    
    # Remover colunas extras para igualar às features do treino
    colunas_para_ignorar = ["risco_defasagem_atual", "ano_pede"]
    X_pred = df_processado.drop(columns=[c for c in colunas_para_ignorar if c in df_processado.columns])
    
    # 4. Fazer Predições
    probabilidades = modelo.predict_proba(X_pred)[:, 1] # Probabilidade da classe 1 (Risco)
    
    # =========================================================================
    # 5. MAPEAMENTO PARA OS NOMES E ORDEM DO APP (SEM NOME E RA)
    # =========================================================================
    
    mapa_colunas = {
        'idade': 'Idade',
        'genero': 'Gênero',
        'fase_ideal': 'Fase Ideal',
        'mat': 'Matemática (MAT)',
        'por': 'Português (POR)',
        'ing': 'Inglês (ING)',
        'iaa': 'Ind. Autoavaliação (IAA)',
        'ieg': 'Ind. Engajamento (IEG)',
        'inde_2024': 'INDE Atual',
        'ips': 'Ind. Psicossocial (IPS)',
        'ipp': 'Ind. Psicopedagógico (IPP)',
        'inde_2022': 'INDE de 2 anos atrás',
        'inde_2023': 'INDE do ano passado'
    }
    
    # Filtra só as colunas que existem e as copia
    colunas_existentes = [c for c in mapa_colunas.keys() if c in df_raw.columns]
    df_validacao = df_raw[colunas_existentes].copy()
    
    # Renomeia para ficar igual ao App
    df_validacao.rename(columns=mapa_colunas, inplace=True)
    
    # Adiciona colunas do Modelo
    df_validacao['Threshold do Modelo'] = round(limiar, 2)
    df_validacao['Probabilidade de Risco'] = (probabilidades * 100).round(1).astype(str) + '%'
    
    # Adiciona as mensagens de resultado
    resultados = []
    recomendacoes = []
    
    for prob in probabilidades:
        if prob >= limiar:
            resultados.append("⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO")
            recomendacoes.append("👉 Recomendação: Necessário acompanhamento pedagógico e psicossocial intensificado.")
        else:
            resultados.append("✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO)")
            recomendacoes.append("👉 Recomendação: Manter acompanhamento padrão para garantir o engajamento.")
            
    df_validacao['Resultado da Análise'] = resultados
    df_validacao['Recomendação App'] = recomendacoes
    
    # Define a ordem exata das colunas baseada na tela do app (Sem Nome e RA)
    ordem_final_tela = [
        'Idade', 'Gênero', 'Fase Ideal', # 1. Dados do Aluno
        'Matemática (MAT)', 'Português (POR)', 'Inglês (ING)', # 2. Notas Acadêmicas
        'Ind. Autoavaliação (IAA)', 'Ind. Engajamento (IEG)', 'INDE Atual', # 3. Indicadores Col 1
        'Ind. Psicossocial (IPS)', 'Ind. Psicopedagógico (IPP)', # 3. Indicadores Col 2
        'INDE de 2 anos atrás', 'INDE do ano passado', # Histórico
        'Threshold do Modelo', 'Probabilidade de Risco', 'Resultado da Análise', 'Recomendação App' # Saídas
    ]
    
    # Garante que só vai ordenar colunas que realmente existem no df
    ordem_final_tela = [c for c in ordem_final_tela if c in df_validacao.columns]
    df_validacao = df_validacao[ordem_final_tela]
    
    # 6. Salvar em Excel
    excel_saida = os.path.join(processed_dir, "validacao_app_passos_magicos.xlsx")
    df_validacao.to_excel(excel_saida, index=False)
    
    print(f"✅ Arquivo gerado com sucesso!")
    print("Foram selecionados 5 registros aleatoriamente.")
    
    # Mostra o resultado final na tela
    return df_validacao

# Executando a função
df_previa = gerar_planilha_validacao()
display(df_previa)

Iniciando a geração da planilha de validação...
✅ Arquivo gerado com sucesso!
Foram selecionados 5 registros aleatoriamente.


,Idade,Gênero,Fase Ideal,Matemática (MAT),Português (POR),Inglês (ING),Ind. Autoavaliação (IAA),Ind. Engajamento (IEG),INDE Atual,Ind. Psicossocial (IPS),Ind. Psicopedagógico (IPP),INDE de 2 anos atrás,INDE do ano passado,Threshold do Modelo,Probabilidade de Risco,Resultado da Análise,Recomendação App
1140,20,Masculino,Fase 8 (Universitários),NaN,NaN,NaN,NaN,0.000000,INCLUIR,NaN,NaN,NaN,NaN,0.45,77.5%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
3007,9,Menino,Fase 1 (4º ano),6.6,5.9,NaN,10.000,7.400000,NaN,7.50,NaN,7.46100,NaN,0.45,97.8%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
606,14,Masculino,Fase 4 (9° ano),8.5,3.5,10.0,9.585,9.382456,7.843158,8.13,7.291667,8.07935,8.095700,0.45,58.0%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
2027,1900-01-16 00:00:00,Feminino,Fase 6 (2° EM),7.7,4.8,9.0,7.500,9.600000,NaN,2.52,8.750000,8.92100,7.289333,0.45,77.5%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
1275,1900-01-11 00:00:00,Feminino,Fase 2 (5° e 6° ano),5.0,5.7,NaN,9.000,10.000000,NaN,7.52,7.500000,NaN,7.974200,0.45,91.2%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
918,15,Feminino,Fase 5 (1° EM),9.5,2.0,10.0,7.917,8.782407,8.04189,7.51,7.343750,NaN,NaN,0.45,69.0%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
1142,21,Feminino,Fase 8 (Universitários),NaN,NaN,NaN,NaN,0.000000,INCLUIR,NaN,NaN,NaN,NaN,0.45,68.0%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
1844,13,Feminino,Fase 3 (7° e 8° ano),5.8,5.2,7.7,8.800,8.900000,NaN,2.52,7.031250,7.55600,7.373892,0.45,5.2%,✅ ALUNO NO CAMINHO CERTO (BAIXO RISCO),👉 Recomendação: Manter acompanhamento padrão p...
2030,17,Masculino,Fase 7 (3° EM),4.5,4.8,6.2,7.500,7.200000,NaN,5.00,7.187500,6.58100,6.293083,0.45,94.2%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
2598,13,Menino,Fase 3 (7º e 8º ano),2.8,2.5,NaN,10.000,5.300000,NaN,7.50,NaN,5.81100,NaN,0.45,96.0%,⚠️ ALTO RISCO DE DEFASAGEM IDENTIFICADO,👉 Recomendação: Necessário acompanhamento peda...
